In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings('ignore')
from arch.unitroot import PhillipsPerron
from scipy.stats import norm
from sklearn.metrics import r2_score

In [2]:
def_data = pd.read_csv("DefRate-Quarterly22.csv")

In [3]:
avg = np.mean(def_data['DefRate'])
def_data['log_CI'] = np.log(def_data['DefRate']) - np.log(avg)
def_data['logit_CI'] = np.log(def_data['DefRate']/(1-def_data['DefRate'])) - np.log(avg/(1-avg))
def_data['probit_CI'] = norm.ppf(def_data['DefRate'])-norm.ppf(avg)

In [4]:
mev = pd.read_csv("\UAE MEV Jan22 v2.csv")
subset_cols = [x for x in mev.columns if ("S1") not in x ]
subset_cols = [x for x in subset_cols if ("S3") not in x ]
mev_df = mev[subset_cols]

In [5]:
model_data = pd.merge(def_data,mev_df,on = 'YrQuarter',how='left')

In [6]:
model_data.head()

,YrQuarter,DefRate,log_CI,logit_CI,probit_CI,OilPrice,NominalGDPBill,UnemployRate,EquityIndex,DebtToGDP,IndProdIndex,CurrAccToGDP,CPI,EIBOR3M,PPI,HPI,RealHPI,GDPBill
0,2008Q4,0.035510,0.554292,0.569838,0.240213,57.547692,1137.800006,4.108209,2390.01,11.926268,104.434728,6.043681,95.093333,NaN,104.434728,147.310502,148.662182,325.255260
1,2009Q1,0.037301,0.603489,0.620893,0.262655,45.683810,1041.624075,4.165888,2487.92,16.218479,96.439725,4.705116,94.450000,NaN,96.439725,133.671750,134.530544,317.490725
2,2009Q2,0.049531,0.887073,0.917262,0.396143,59.860625,944.682562,4.223250,2631.32,20.812280,94.940662,3.301876,93.593333,NaN,94.940662,124.809195,126.123910,311.434061
3,2009Q3,0.046061,0.814423,0.840967,0.361244,68.828030,915.223952,4.196574,3124.22,23.021086,95.940037,2.158828,94.416667,NaN,95.940037,115.683436,117.096130,309.949323
4,2009Q4,0.018304,-0.108424,-0.110561,-0.044556,75.521692,964.774573,4.079907,2743.61,21.806943,98.438476,1.589596,94.920000,1.937831,98.438476,113.896277,115.060357,315.473074


In [7]:
transform_cols = list(model_data.columns[5:])

In [8]:
def feature_transform(data,cols):
    for col in cols:
        # Log Transformation
        data['log_'+col] = np.log(data[col])
        #Lag 
        for i in range(1,7):
            data['lag_'+str(i)+'_'+col] = data[col].shift(i)
        #Diff
        for i in range(1,7):
            data['diff_'+str(i)+"Q_"+col] = data[col] - data[col].shift(i)
        # Per Diff
        for i in range(1,7):
            data['Per_diff_'+str(i)+"Q_"+col] = (data[col]/data[col].shift(i))-1
        # log_dff
        data['log_diff_'+col] = np.log(data[col]) - np.log(data[col].shift(1))

        # Log Diff Lag 
        for i in range(1,7):
            data['log_diff_lag_'+str(i)+'_'+col] = data['log_diff_'+col].shift(i)

        for i in range(1,7):
            data['diff_lag_'+str(i)+"Q_"+col] =  data['diff_'+str(i)+"Q_"+col].shift(i)

    return data

In [9]:
transformed_data = feature_transform(model_data,transform_cols)

In [10]:
transformed_data.head()

,YrQuarter,DefRate,log_CI,logit_CI,probit_CI,OilPrice,NominalGDPBill,UnemployRate,EquityIndex,DebtToGDP,IndProdIndex,CurrAccToGDP,CPI,EIBOR3M,PPI,HPI,RealHPI,GDPBill,log_OilPrice,lag_1_OilPrice,lag_2_OilPrice,lag_3_OilPrice,lag_4_OilPrice,lag_5_OilPrice,lag_6_OilPrice,diff_1Q_OilPrice,diff_2Q_OilPrice,diff_3Q_OilPrice,diff_4Q_OilPrice,diff_5Q_OilPrice,diff_6Q_OilPrice,Per_diff_1Q_OilPrice,Per_diff_2Q_OilPrice,Per_diff_3Q_OilPrice,Per_diff_4Q_OilPrice,Per_diff_5Q_OilPrice,Per_diff_6Q_OilPrice,log_diff_OilPrice,log_diff_lag_1_OilPrice,log_diff_lag_2_OilPrice,log_diff_lag_3_OilPrice,log_diff_lag_4_OilPrice,log_diff_lag_5_OilPrice,log_diff_lag_6_OilPrice,diff_lag_1Q_OilPrice,diff_lag_2Q_OilPrice,diff_lag_3Q_OilPrice,diff_lag_4Q_OilPrice,diff_lag_5Q_OilPrice,diff_lag_6Q_OilPrice,log_NominalGDPBill,lag_1_NominalGDPBill,lag_2_NominalGDPBill,lag_3_NominalGDPBill,lag_4_NominalGDPBill,lag_5_NominalGDPBill,lag_6_NominalGDPBill,diff_1Q_NominalGDPBill,diff_2Q_NominalGDPBill,diff_3Q_NominalGDPBill,diff_4Q_NominalGDPBill,diff_5Q_NominalGDPBill,diff_6Q_NominalGDPBill,Per_diff_1Q_NominalGDPBill,Per_diff_2Q_NominalGDPBill,Per_diff_3Q_NominalGDPBill,Per_diff_4Q_NominalGDPBill,Per_diff_5Q_NominalGDPBill,Per_diff_6Q_NominalGDPBill,log_diff_NominalGDPBill,log_diff_lag_1_NominalGDPBill,log_diff_lag_2_NominalGDPBill,log_diff_lag_3_NominalGDPBill,log_diff_lag_4_NominalGDPBill,log_diff_lag_5_NominalGDPBill,log_diff_lag_6_NominalGDPBill,diff_lag_1Q_NominalGDPBill,diff_lag_2Q_NominalGDPBill,diff_lag_3Q_NominalGDPBill,diff_lag_4Q_NominalGDPBill,diff_lag_5Q_NominalGDPBill,diff_lag_6Q_NominalGDPBill,log_UnemployRate,lag_1_UnemployRate,lag_2_UnemployRate,lag_3_UnemployRate,lag_4_UnemployRate,lag_5_UnemployRate,lag_6_UnemployRate,diff_1Q_UnemployRate,diff_2Q_UnemployRate,diff_3Q_UnemployRate,diff_4Q_UnemployRate,diff_5Q_UnemployRate,diff_6Q_UnemployRate,Per_diff_1Q_UnemployRate,Per_diff_2Q_UnemployRate,Per_diff_3Q_UnemployRate,Per_diff_4Q_UnemployRate,Per_diff_5Q_UnemployRate,Per_diff_6Q_UnemployRate,log_diff_UnemployRate,log_diff_lag_1_UnemployRate,log_diff_lag_2_UnemployRate,log_diff_lag_3_UnemployRate,log_diff_lag_4_UnemployRate,log_diff_lag_5_UnemployRate,log_diff_lag_6_UnemployRate,diff_lag_1Q_UnemployRate,diff_lag_2Q_UnemployRate,diff_lag_3Q_UnemployRate,diff_lag_4Q_UnemployRate,diff_lag_5Q_UnemployRate,diff_lag_6Q_UnemployRate,log_EquityIndex,lag_1_EquityIndex,lag_2_EquityIndex,lag_3_EquityIndex,lag_4_EquityIndex,lag_5_EquityIndex,lag_6_EquityIndex,diff_1Q_EquityIndex,diff_2Q_EquityIndex,diff_3Q_EquityIndex,diff_4Q_EquityIndex,diff_5Q_EquityIndex,diff_6Q_EquityIndex,Per_diff_1Q_EquityIndex,Per_diff_2Q_EquityIndex,Per_diff_3Q_EquityIndex,Per_diff_4Q_EquityIndex,Per_diff_5Q_EquityIndex,Per_diff_6Q_EquityIndex,log_diff_EquityIndex,log_diff_lag_1_EquityIndex,log_diff_lag_2_EquityIndex,log_diff_lag_3_EquityIndex,log_diff_lag_4_EquityIndex,log_diff_lag_5_EquityIndex,log_diff_lag_6_EquityIndex,diff_lag_1Q_EquityIndex,diff_lag_2Q_EquityIndex,diff_lag_3Q_EquityIndex,diff_lag_4Q_EquityIndex,diff_lag_5Q_EquityIndex,diff_lag_6Q_EquityIndex,log_DebtToGDP,lag_1_DebtToGDP,lag_2_DebtToGDP,lag_3_DebtToGDP,...,diff_4Q_EIBOR3M,diff_5Q_EIBOR3M,diff_6Q_EIBOR3M,Per_diff_1Q_EIBOR3M,Per_diff_2Q_EIBOR3M,Per_diff_3Q_EIBOR3M,Per_diff_4Q_EIBOR3M,Per_diff_5Q_EIBOR3M,Per_diff_6Q_EIBOR3M,log_diff_EIBOR3M,log_diff_lag_1_EIBOR3M,log_diff_lag_2_EIBOR3M,log_diff_lag_3_EIBOR3M,log_diff_lag_4_EIBOR3M,log_diff_lag_5_EIBOR3M,log_diff_lag_6_EIBOR3M,diff_lag_1Q_EIBOR3M,diff_lag_2Q_EIBOR3M,diff_lag_3Q_EIBOR3M,diff_lag_4Q_EIBOR3M,diff_lag_5Q_EIBOR3M,diff_lag_6Q_EIBOR3M,log_PPI,lag_1_PPI,lag_2_PPI,lag_3_PPI,lag_4_PPI,lag_5_PPI,lag_6_PPI,diff_1Q_PPI,diff_2Q_PPI,diff_3Q_PPI,diff_4Q_PPI,diff_5Q_PPI,diff_6Q_PPI,Per_diff_1Q_PPI,Per_diff_2Q_PPI,Per_diff_3Q_PPI,Per_diff_4Q_PPI,Per_diff_5Q_PPI,Per_diff_6Q_PPI,log_diff_PPI,log_diff_lag_1_PPI,log_diff_lag_2_PPI,log_diff_lag_3_PPI,log_diff_lag_4_PPI,log_diff_lag_5_PPI,log_diff_lag_6_PPI,diff_lag_1Q_PPI,diff_lag_2Q_PPI,diff_lag_3Q_PPI,diff_lag_4Q_PPI,diff_l

In [13]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tsa.stattools import adfuller,kpss

In [ ]:
count = 0
stationarity_test = pd.DataFrame(columns=['ColName','ADF_Status','KPSS_Status','PP_Status','Overall'])
for col in model_data.columns[1:]:
    adf_result = adfuller(model_data[col].dropna())
    adf_status = 'Stationary' if adf_result[1]<0.05 else 'Non-Stationary'
    adf_val = 1 if adf_result[1]<0.05 else 0

    kpss_stat,kpss_pval,kpss_nlag,kpss_cv = kpss(model_data[col].dropna())
    kpss_status = 'Non-Stationary' if kpss_pval<0.05 else 'Stationary'
    kpss_val = 0 if kpss_pval<0.05 else 1

    pp = PhillipsPerron(model_data[col].dropna())
    pp_status = 'Stationary' if pp.pvalue<0.05 else 'Non-Stationary'
    pp_val = 1 if pp.pvalue<0.05 else 0

    overall = 'Stationary' if (adf_val+kpss_val+pp_val)>=2 else 'Non-Stationary'

    stationarity_test.loc[count] = [col,adf_status,kpss_status,pp_status,overall]
    count+=1


In [15]:
stationary_cols = list(stationarity_test[stationarity_test['Overall']=='Stationary']['ColName'])
all_cols = list(model_data.columns[5:])

In [16]:
model_data.index = np.arange(len(model_data))

In [17]:
train_data = model_data.iloc[:model_data[model_data['YrQuarter']=='2018Q4'].index.values[0]+1]
test_data = model_data.iloc[model_data[model_data['YrQuarter']=='2018Q4'].index.values[0]+1:model_data[model_data['YrQuarter']=='2020Q2'].index.values[0]+1]

# Logit CI

In [19]:
output_col = 'logit_CI'
y = train_data[output_col]
X = train_data[stationary_cols]

In [20]:
function_dict = {'predictor': [], 'r-squared':[]}
#Iterate through every column in X
for col in X.columns:
    #Create a dataframe called selected_X with only the 1 column
    data = train_data[[col,output_col]].dropna()
    selected_X = data[[col]]
    selected_Y = data[output_col]
    #Fit a model for our target and our selected column 
    model = sm.OLS(selected_Y, sm.add_constant(selected_X)).fit()
    #Predict what our target would be for our model
    y_preds = model.predict(sm.add_constant(selected_X))
    #Add the column name to our dictionary
    function_dict['predictor'].append(col)
    #Calculate the r-squared value between the target and predicted target
    r2 = np.corrcoef(selected_Y, y_preds)[0, 1]**2
    #Add the r-squared value to our dictionary
    function_dict['r-squared'].append(r2)
    
#Once it's iterated through every column, turn our dictionary into a DataFrame and sort it
function_df = pd.DataFrame(function_dict).sort_values(by=['r-squared'], ascending = False)
#Display only the top 5 predictors


In [21]:
function_df.head(10)

,predictor,r-squared
124,Per_diff_5Q_EIBOR3M,0.573151
167,lag_6_RealHPI,0.556448
123,Per_diff_3Q_EIBOR3M,0.549446
122,Per_diff_2Q_EIBOR3M,0.494866
166,lag_5_RealHPI,0.427014
121,Per_diff_1Q_EIBOR3M,0.404661
125,log_diff_EIBOR3M,0.397237
126,log_diff_lag_1_EIBOR3M,0.392910
59,log_diff_lag_1_DebtToGDP,0.369823
38,diff_6Q_EquityIndex,0.360948


In [22]:
def next_possible_feature (data,all_features, current_features, ignore_features=[]):
    '''
    This function will loop through each column that isn't in your feature model and 
    calculate the r-squared value if it were the next feature added to your model. 
    It will display a dataframe with a sorted r-squared value.
    X_npf = X dataframe
    y_npf = y dataframe
    current_features = list of features that are already in your model
    ignore_features = list of unused features we want to skip over
    '''   
    #Create an empty dictionary that will be used to store our results
    function_dict = {'predictor': [], 'r-squared':[]}
    #Iterate through every column in X
    for col in all_features:
        #But only create a model if the feature isn't already selected or ignored
        if col not in (current_features+ignore_features):
            #Create a dataframe called function_X with our current features + 1
            t_data =  data[current_features + [col,output_col]].dropna()
            
            selected_X = t_data[col]
            selected_Y = t_data[output_col].values

            #Fit a model for our target and our selected columns 
            model = sm.OLS(selected_Y, sm.add_constant(selected_X)).fit()
            #Predict what  our target would be for our selected columns
            y_preds = model.predict(sm.add_constant(selected_X))
            #Add the column name to our dictionary
            function_dict['predictor'].append(col)
            #Calculate the r-squared value between the target and predicted target
            r2 = np.corrcoef(selected_Y, y_preds)[0, 1]**2
            #Add the r-squared value to our dictionary
            function_dict['r-squared'].append(r2)
    #Once it's iterated through every column, turn our dict into a sorted DataFrame
    function_df = pd.DataFrame(function_dict).sort_values(by=['r-squared'],ascending = False)

    return function_df

In [543]:
from mlxtend.feature_selection import SequentialFeatureSelector


In [544]:
sfs = SequentialFeatureSelector(LinearRegression(), forward=True, k_features=4)

In [545]:
temp_data = train_data.dropna()

In [546]:
sfs.fit(temp_data[stationary_cols], temp_data['log_CI'])

SequentialFeatureSelector(estimator=LinearRegression(), k_features=4)

In [547]:
final_selected_cols = final_feat
final_df =  train_data[final_selected_cols+[output_col]].dropna()

selected_X = final_df[final_selected_cols]
selected_Y = final_df[output_col].values

selected_X = sm.add_constant(selected_X)


In [550]:
mod = sm.OLS(selected_Y, selected_X)    # Describe model

res = mod.fit()       # Fit model

print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.568
Model:                            OLS   Adj. R-squared:                  0.541
Method:                 Least Squares   F-statistic:                     21.05
Date:                Mon, 25 Jul 2022   Prob (F-statistic):           1.46e-06
Time:                        10:30:16   Log-Likelihood:                -18.321
No. Observations:                  35   AIC:                             42.64
Df Residuals:                      32   BIC:                             47.31
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     